In [12]:
import os
import tkinter, tkinter.filedialog#[開く]で元画像フォルダー選択するために必要なライブラリ
from PIL import Image, ImageDraw, ImageFilter#画像処理に必要なライブラリ
from PIL import Image, ImageDraw, ImageFont#画像に文字を入れるために必要なライブラリ
import cv2
import math
import numpy as np
from PIL import Image
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')#学習データ

#画像フォルダー選択
root = tkinter.Tk()
root.withdraw()
msg = '画像フォルダーを選択してください。'
img_dir_path = tkinter.filedialog.askdirectory(title=msg)
if (not img_dir_path): #[キャンセル]クリック時の処理
    print('フォルダーを選んでください。')
    sys.exit()
#画像フォルダー選択ここまで

print(img_dir_path)

cover_folder = os.path.join(img_dir_path,'cover')
other_folder = os.path.join(img_dir_path,'other')


cover_list = os.listdir(cover_folder)
if '.DS_Store' in cover_list:
    cover_list.remove('.DS_Store')
#エラー回避処理→結果、cover_list[0]が表紙のJPG画像となる

other_list = os.listdir(other_folder)
other_list = sorted(other_list)#番号が若い順にソート
if '.DS_Store' in cover_list:
    cover_list.remove('.DS_Store')
#エラー回避処理→結果、other_list[0]が000.jpgとなる



def single1(i,x,n):#片ページに１枚、片ページは空白の見開きをページを定義(最初や最後のページに使用)
    im = Image.open(os.path.join(other_folder,other_list[i]))
    im_width, im_height = im.size
    base = Image.new("RGB", (2000, 1330), 'white')#下地の白画像をbaseという名前で生成
    if im_width < im_height:
        resize_im = im.resize((886, 1330))#縦
        base.paste(resize_im, (int(x), 0))#縦  
        base.save(os.path.join(img_dir_path, "000"+str(n)+".jpg"), quality=100, dpi=(300, 300))
    elif im_width > im_height:
        resize_im = im.resize((1000, 666))#横
        base.paste(resize_im, (int(x), 332))#横
        base.save(os.path.join(img_dir_path, "000"+str(n)+".jpg"), quality=100, dpi=(300, 300))
        
        
def both2(i,n):#両開きで2枚の画像を使うページを定義
    
    base = Image.new("RGB", (2000, 1330), 'white')#下地の白画像をbaseという名前で生成    
    #左ページに大きく貼り付ける
    im0 = Image.open(os.path.join(other_folder,other_list[i]))
    im0_width, im0_height = im0.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im0,(int((4080-int(im0_width))/2),int((4080-int(im0_height))/2)))
    resize_base_sq = base_sq.resize((1000,1000))
    base.paste(resize_base_sq, (0, int((1330-1000)/2))) 
    #右ページ上に貼り付ける
    im1 = Image.open(os.path.join(other_folder,other_list[i+1]))
    im1_width, im1_height = im1.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im1,(int((4080-int(im1_width))/2),int((4080-int(im1_height))/2)))
    resize_base_sq = base_sq.resize((1000,1000))
    base.paste(resize_base_sq, (1000, int((1330-1000)/2))) 
    
    base.save(os.path.join(img_dir_path, "000"+str(n)+".jpg"), quality=100, dpi=(300, 300))

        
def both3l(i,n):#両開きで３枚の画像を使うページを定義
    
    def face_width(i):    
        img = cv2.imread(os.path.join(other_folder,other_list[i]))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.2, minSize = (250,250), maxSize = (1600,1600))
    
        if  len(faces) > 0:
            face_max = faces[faces[:, 2].argmax()]
            face_inorder.append(other_list[i])
            values.append(face_max[2])
    
        elif len(faces) == 0:#顔認証に失敗した場合
            img = cv2.imread(os.path.join(other_folder,other_list[i]))
            for j in range(-17,17):#画像を回転
                angle = 5*j
                scale = 1.0
                height, width = img.shape[:2]
    #回転の中心を指定                          
                center = (int(width/2), int(height/2))      
                trans = cv2.getRotationMatrix2D(center, angle , scale)
    #アフィン変換
                img_rot = cv2.warpAffine(img, trans, (width,height))      
                gray = cv2.cvtColor(img_rot, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.2, minSize = (250,250), maxSize = (1600,1600))
                if len(faces) > 0:
                    face_max = faces[faces[:, 2].argmax()]
                    face_inorder.append(other_list[i])
                    values.append(face_max[2])
                    break
            
                elif len(faces) == 0:#上記処理でも認識できなかったら
                    face_inorder.append(other_list[i])
                    values.append(0)
#顔認証の定義ここまで
    
    
    
    face_inorder = []
    values = []
    j = i + 2
    while i <= j:    
        face_width(i)
        i = i +1
    dic = dict(zip(face_inorder, values))
    dic = sorted(dic.items(), key=lambda x:x[1])#values(顔の大きさ)順でソート
    
    base = Image.new("RGB", (2000, 1330), 'white')#下地の白画像をbaseという名前で生成
    
    #顔の大きさが最小＝引きの写真を左ページに大きく貼り付ける
    dic0 = dic[0]#辞書の中でキーが最小のものを指定
    im0 = Image.open(os.path.join(other_folder,dic0[0]))#dic0[0]と指定することで画像パスのみに変換
    print(dic[0])
    im0_width, im0_height = im0.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im0,(int((4080-int(im0_width))/2),int((4080-int(im0_height))/2)))
    resize_base_sq = base_sq.resize((1000,1000))
    base.paste(resize_base_sq, (0, int((1330-1000)/2)))#縦  
    
    # 顔の大きさが２番目の写真を右ページ上に貼り付ける
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    dic1 = dic[1]#辞書の中でキーが２番目の大きさのものを指定
    im1 = Image.open(os.path.join(other_folder,dic1[0]))
    im1_width, im1_height = im1.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im1,(int((4080-int(im1_width))/2),int((4080-int(im1_height))/2)))
    resize_base_sq = base_sq.resize((665,665))#1330÷2
    base.paste(resize_base_sq, (1167, 0)) 
    
    # 顔の大きさが３番目の写真を右ページに下に貼り付ける
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    dic2 = dic[2]#辞書の中でキーが２番目の大きさのものを指定
    im2 = Image.open(os.path.join(other_folder,dic2[0]))
    im2_width, im2_height = im2.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im2,(int((4080-int(im2_width))/2),int((4080-int(im2_height))/2)))
    resize_base_sq = base_sq.resize((665,665))#1330÷2
    base.paste(resize_base_sq, (1167, 665))    
    
    base.save(os.path.join(img_dir_path, "000"+str(n)+".jpg"), quality=100, dpi=(300, 300))


    

    
def both5l(i,n):#両開きで5枚の画像を使うページを定義
    
    def face_width(i):    
        img = cv2.imread(os.path.join(other_folder,other_list[i]))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.2, minSize = (250,250), maxSize = (1600,1600))
    
        if  len(faces) > 0:
            face_max = faces[faces[:, 2].argmax()]
            face_inorder.append(other_list[i])
            values.append(face_max[2])
    
        elif len(faces) == 0:#顔認証に失敗した場合
            img = cv2.imread(os.path.join(other_folder,other_list[i]))
            for j in range(-17,17):#画像を回転
                angle = 5*j
                scale = 1.0
                height, width = img.shape[:2]
    #回転の中心を指定                          
                center = (int(width/2), int(height/2))      
                trans = cv2.getRotationMatrix2D(center, angle , scale)
    #アフィン変換
                img_rot = cv2.warpAffine(img, trans, (width,height))      
                gray = cv2.cvtColor(img_rot, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.2, minSize = (250,250), maxSize = (1600,1600))
                if len(faces) > 0:
                    face_max = faces[faces[:, 2].argmax()]
                    face_inorder.append(other_list[i])
                    values.append(face_max[2])
                    break
            
                elif len(faces) == 0:#上記処理でも認識できなかったら
                    face_inorder.append(other_list[i])
                    values.append(0)
#顔認証の定義ここまで
    
    
    
    face_inorder = []
    values = []
    j = i + 4
    while i <= j:    
        face_width(i)
        i = i +1
    dic = dict(zip(face_inorder, values))
    dic = sorted(dic.items(), key=lambda x:x[1])#values(顔の大きさ)順でソート
    
    base = Image.new("RGB", (2000, 1330), 'white')#下地の白画像をbaseという名前で生成
    
    #顔の大きさが最小＝引きの写真を左ページに大きく貼り付ける
    dic0 = dic[0]#辞書の中でキーが最小のものを指定
    print(dic[0])
    im0 = Image.open(os.path.join(other_folder,dic0[0]))#dic0[0]と指定することで画像パスのみに変換
    im0_width, im0_height = im0.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im0,(int((4080-int(im0_width))/2),int((4080-int(im0_height))/2)))
    resize_base_sq = base_sq.resize((1000,1000))
    base.paste(resize_base_sq, (0, int((1330-1000)/2)))#縦  
    
    # 顔の大きさが２番目の写真を右ページに貼り付ける
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    dic1 = dic[1]#辞書の中でキーが２番目の大きさのものを指定
    im1 = Image.open(os.path.join(other_folder,dic1[0]))
    im1_width, im1_height = im1.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im1,(int((4080-int(im1_width))/2),int((4080-int(im1_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (1000, 165)) 
    
    # 顔の大きさが３番目の写真を右ページに貼り付ける
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    dic2 = dic[2]#辞書の中でキーが２番目の大きさのものを指定
    im2 = Image.open(os.path.join(other_folder,dic2[0]))
    im2_width, im2_height = im2.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im2,(int((4080-int(im2_width))/2),int((4080-int(im2_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (1500, 165))    
    
        # 顔の大きさが２番目の写真を右ページに貼り付ける
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    dic3 = dic[3]#辞書の中でキーが3番目の大きさのものを指定
    im3 = Image.open(os.path.join(other_folder,dic3[0]))
    im3_width, im3_height = im3.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im3,(int((4080-int(im3_width))/2),int((4080-int(im3_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (1000, 665)) #1330÷2
    
    # 顔の大きさが３番目の写真を右ページに貼り付ける
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    dic4 = dic[4]#辞書の中でキーが4番目の大きさのものを指定
    im4 = Image.open(os.path.join(other_folder,dic4[0]))
    im4_width, im4_height = im4.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im4,(int((4080-int(im4_width))/2),int((4080-int(im4_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (1500, 665))
    
    
    base.save(os.path.join(img_dir_path, "000"+str(n)+".jpg"), quality=100, dpi=(300, 300))
    
    
def both4(i,n):#両開きで４枚の画像を使うページを定義
    base = Image.new("RGB", (2000, 1330), 'white')#下地の白画像をbaseという名前で生成

    # 1番目の写真を左ページ上に貼り付ける
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im1 = Image.open(os.path.join(other_folder,other_list[i]))
    im1_width, im1_height = im1.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im1,(int((4080-int(im1_width))/2),int((4080-int(im1_height))/2)))
    resize_base_sq = base_sq.resize((665,665))#1330÷2
    base.paste(resize_base_sq, (167, 0)) 
    
    #2番目の写真を左ページに下に貼り付ける
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im2 = Image.open(os.path.join(other_folder,other_list[i+1]))
    im2_width, im2_height = im2.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im2,(int((4080-int(im2_width))/2),int((4080-int(im2_height))/2)))
    resize_base_sq = base_sq.resize((665,665))#1330÷2
    base.paste(resize_base_sq, (167, 665))    


    #３番目の写真を右ページ上に貼り付ける
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im3 = Image.open(os.path.join(other_folder,other_list[i+2]))
    im3_width, im3_height = im3.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im3,(int((4080-int(im3_width))/2),int((4080-int(im3_height))/2)))
    resize_base_sq = base_sq.resize((665,665))#1330÷2
    base.paste(resize_base_sq, (1167, 0)) 
    
    #４番目の写真を右ページに下に貼り付ける
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im4 = Image.open(os.path.join(other_folder,other_list[i+3]))
    im4_width, im4_height = im4.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im4,(int((4080-int(im4_width))/2),int((4080-int(im4_height))/2)))
    resize_base_sq = base_sq.resize((665,665))#1330÷2
    base.paste(resize_base_sq, (1167, 665))    
    
    base.save(os.path.join(img_dir_path, "000"+str(n)+".jpg"), quality=100, dpi=(300, 300))
    
    
def both8(i,n):#両開きで5枚の画像を使うページを定義
    base = Image.new("RGB", (2000, 1330), 'white')#下地の白画像をbaseという名前で生成
    
    #1番目の写真
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im1 = Image.open(os.path.join(other_folder,other_list[i]))
    im1_width, im1_height = im1.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im1,(int((4080-int(im1_width))/2),int((4080-int(im1_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (0, 165)) 
    
    #2番目の写真
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im2 = Image.open(os.path.join(other_folder,other_list[i+1]))
    im2_width, im2_height = im2.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im2,(int((4080-int(im2_width))/2),int((4080-int(im2_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (500, 165))    
    
    #3番目の写真
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im3 = Image.open(os.path.join(other_folder,other_list[i+2]))
    im3_width, im3_height = im3.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im3,(int((4080-int(im3_width))/2),int((4080-int(im3_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (0, 665)) #1330÷2
    
    #4番目の写真
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im4 = Image.open(os.path.join(other_folder,other_list[i+3]))
    im4_width, im4_height = im4.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im4,(int((4080-int(im4_width))/2),int((4080-int(im4_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (500, 665))
    
    
    #5番目の写真
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im5 = Image.open(os.path.join(other_folder,other_list[i+4]))
    im5_width, im5_height = im5.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im5,(int((4080-int(im5_width))/2),int((4080-int(im5_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (1000, 165)) 
    
    #6番目の写真
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im6 = Image.open(os.path.join(other_folder,other_list[i+5]))
    im6_width, im6_height = im6.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im6,(int((4080-int(im6_width))/2),int((4080-int(im6_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (1500, 165))    
    
    #7番目の写真
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im7 = Image.open(os.path.join(other_folder,other_list[i+6]))
    im7_width, im7_height = im7.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im7,(int((4080-int(im7_width))/2),int((4080-int(im7_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (1000, 665)) #1330÷2
    
    #8番目の写真
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    im8 = Image.open(os.path.join(other_folder,other_list[i+7]))
    im8_width, im8_height = im8.size
    base_sq =  Image.new("RGB", (4080, 4080), 'white')
    base_sq.paste(im8,(int((4080-int(im8_width))/2),int((4080-int(im8_height))/2)))
    resize_base_sq = base_sq.resize((500,500))#1000÷2
    base.paste(resize_base_sq, (1500, 665))
    
    
    base.save(os.path.join(img_dir_path, "000"+str(n)+".jpg"), quality=100, dpi=(300, 300))

    
#■■■ここから表紙の処理■■
base = Image.new("RGB", (2000, 1330), 'white')#下地の白画像をbaseという名前で生成    
im1 = Image.open(os.path.join(cover_folder,cover_list[0]))
im1_width, im1_height = im1.size
base_sq =  Image.new("RGB", (4080, 4080), 'white')
base_sq.paste(im1,(int((4080-int(im1_width))/2),int((4080-int(im1_height))/2)))
resize_base_sq = base_sq.resize((1000,1000))
base.paste(resize_base_sq, (1000, int((1330-1000)/2))) 
base.save(os.path.join(img_dir_path, "0000.jpg"), quality=100, dpi=(300, 300))

#■■■ここから内ページの処理■■
#1枚目
single1(1,1000,1)#(i,x,n)＝(写真のi番目,ペーストするX座標,ファイル名の数字n)
#2〜4枚目
both3l(2,2)#(i,n)＝(写真のi番目,ファイル名の数字n)
#5~9枚目
both5l(5,3)
#10~13枚目
both4(10,4)
#14~21枚目
both8(14,5)
#22〜24枚目
both3l(22,6)
#25~26枚目
both2(25,7)
#27~30枚目
both4(27,8)
#31~35枚目
both5l(31,9)
#36~39枚目
both4(36,10)
#40~41枚目
both2(40,11)
#42~49枚目
both8(42,12)
#50〜52枚目
both3l(50,13)
#53~56枚目
both4(53,14)
#57~63枚目
both8(57,15)
#65~69枚目
both5l(65,16)
#70~73枚目
both4(70,17)
#74枚目
single1(74,0,18)

print("END")

/Users/user/Desktop/sample
('20200725y06_1520604_001.jpg', 345)
('20200725y06_1520604_004.jpg', 280)
('20200725y06_1520604_024.jpg', 479)
('20200725y06_1520604_033.jpg', 298)
('20200725y06_1520604_051.jpg', 469)
('20200725y06_1520604_065.jpg', 381)
END
